In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [10]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split

# 1. Define Preprocessing Pipeline
# The standard preprocessing for deep learning models is scaling to [0, 1] (via ToTensor) 
# and then Z-score normalization using known means/stds (often ImageNet values for transfer learning)
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.Resize(256),         # Resize all images to 256x256
    transforms.CenterCrop(224),     # Crop to 224x224 (standard input size for many CNNs)
    transforms.ToTensor(),          # Converts image to a PyTorch Tensor and scales to [0, 1]
    transforms.Normalize(mean=MEAN, std=STD) # Apply normalization
])

# 2. Load the Dataset
data_dir = '../data/vehicle-type-detecion' # Assuming you run the script from the parent directory
full_dataset = datasets.ImageFolder(data_dir, transform=transform)

# 3. Split the Data
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# 4. Create DataLoaders
# DataLoader handles batching, shuffling, and multi-threaded loading
BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [13]:
import numpy as np
import torch.nn as nn
from torchvision import models
from tqdm import tqdm

# Load a pre-trained model (e.g., ResNet-18)
feature_extractor = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# Freeze weights (we only want the features, not to train the extractor)
for param in feature_extractor.parameters():
    param.requires_grad = False

# Remove the final classification layer (the last layer before the output)
# The features are the output of the AdaptiveAvgPool2d layer (512 features for ResNet-18)
feature_extractor = nn.Sequential(*(list(feature_extractor.children())[:-1]))
feature_extractor.eval() # Set to evaluation mode

def extract_features(data_loader):
    features_list = []
    labels_list = []
    
    with torch.no_grad(): # No need to calculate gradients for feature extraction
        for inputs, labels in tqdm(data_loader, desc="Extracting Features"):
            # Get the features (shape: [batch_size, 512, 1, 1])
            outputs = feature_extractor(inputs)
            
            # Flatten the features (shape: [batch_size, 512])
            outputs = outputs.squeeze().cpu().numpy()
            
            features_list.append(outputs)
            labels_list.append(labels.cpu().numpy())

    # Concatenate all batches
    X = np.concatenate(features_list)
    Y = np.concatenate(labels_list)
    return X, Y

# Extract features and labels for training and validation
X_train, Y_train = extract_features(train_loader)
X_val, Y_val = extract_features(val_loader)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/liamshepard/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:04<00:00, 9.68MB/s]
Extracting Features: 100%|██████████| 9/9 [00:04<00:00,  2.23it/s]


In [14]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Initialize and train the XGBoost Classifier
xgb_model = xgb.XGBClassifier(
    objective='multi:softmax', 
    num_class=len(full_dataset.classes), 
    use_label_encoder=False, 
    eval_metric='merror',
    n_estimators=100
)

print("Training XGBoost Classifier...")
xgb_model.fit(X_train, Y_train)

# Predict and evaluate
Y_pred_xgb = xgb_model.predict(X_val)
xgb_accuracy = accuracy_score(Y_val, Y_pred_xgb)

print(f"\nXGBoost Validation Accuracy: {xgb_accuracy * 100:.2f}%")

Training XGBoost Classifier...


/Users/liamshepard/Desktop/Skole/semester5/data/DDP/IT3212-Computer-Vision/.venv/lib/python3.13/site-packages/xgboost/training.py:199: UserWarning: [18:03:39] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



XGBoost Validation Accuracy: 87.40%


In [15]:
import torch.optim as optim

# Load a pre-trained ResNet-18 model
model_cnn = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
num_ftrs = model_cnn.fc.in_features # Get the number of input features for the final layer

# Replace the final fully connected layer (model_cnn.fc)
# The output size must match the number of your classes (4: hatchback, motorcycle, pickup, sedan, suv)
# Wait, based on your directory listing, you have 5 classes: hatchback, motorcycle, pickup, sedan, suv
num_classes = len(full_dataset.classes) # Should be 5
model_cnn.fc = nn.Linear(num_ftrs, num_classes)

# Move the model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_cnn = model_cnn.to(device)

# Define Loss function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_cnn.parameters(), lr=0.001, momentum=0.9)

In [16]:
def train_model(model, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Training phase
        model.train()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in tqdm(train_loader, desc=f"Training Epoch {epoch}"):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad() # Zero the parameter gradients

            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_dataset)
        epoch_acc = running_corrects.double() / len(train_dataset)

        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # Validation phase (optional, but highly recommended)
        model.eval()
        # ... (similar loop logic for validation, without backward pass and optimizer step)
        
    return model

# Train the CNN
model_cnn = train_model(model_cnn, criterion, optimizer, num_epochs=5)
print("\nCNN Training Complete.")

Epoch 0/4
----------


Training Epoch 0: 100%|██████████| 33/33 [00:41<00:00,  1.25s/it]


Train Loss: 1.2149 Acc: 0.5506
Epoch 1/4
----------


Training Epoch 1: 100%|██████████| 33/33 [00:40<00:00,  1.24s/it]


Train Loss: 0.5578 Acc: 0.8483
Epoch 2/4
----------


...

Train Loss: 0.3118 Acc: 0.9208
Epoch 3/4
----------


Training Epoch 3: 100%|██████████| 33/33 [00:41<00:00,  1.26s/it]


Train Loss: 0.2020 Acc: 0.9580
Epoch 4/4
----------


Training Epoch 4: 100%|██████████| 33/33 [00:41<00:00,  1.26s/it]

Train Loss: 0.1249 Acc: 0.9885

CNN Training Complete.


In [18]:
import torch
from tqdm import tqdm

def evaluate_model(model, data_loader, criterion, device):
    # Set the model to evaluation mode
    model.eval() 
    
    running_loss = 0.0
    running_corrects = 0
    total_samples = 0
    
    # Disable gradient calculations for inference (saves memory and speeds up computation)
    with torch.no_grad():
        for inputs, labels in tqdm(data_loader, desc="Validating"):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Get predictions
            _, preds = torch.max(outputs.data, 1)

            # Update statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total_samples += inputs.size(0)

    # Calculate final loss and accuracy
    final_loss = running_loss / total_samples
    final_acc = running_corrects.double() / total_samples
    
    return final_loss, final_acc.item()



import torch.nn as nn

# Assuming these variables are defined from your training setup:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# criterion = nn.CrossEntropyLoss()
# model_cnn = ... # your trained model
# val_loader = ... # your validation data loader

final_val_loss, final_val_acc = evaluate_model(
    model=model_cnn,
    data_loader=val_loader,
    criterion=criterion,
    device=device
)

print(f"✅ Final Validation Loss: {final_val_loss:.4f}")
print(f"✅ Final Validation Accuracy: {final_val_acc * 100:.2f}%")

Validating: 100%|██████████| 9/9 [00:03<00:00,  2.34it/s]

✅ Final Validation Loss: 0.2391
✅ Final Validation Accuracy: 93.89%
